# Custom State for Memory Management

This notebook demonstrates how to create custom agent state schemas to track additional information beyond just conversation messages.

## Key Concepts
- **Default AgentState**: Only tracks messages[]
- **Custom AgentState**: Messages + user preferences + session data + task history
- **State Persistence**: Within and between conversations
- **Memory Management**: Context-aware responses based on history

## Benefits of Custom State
- **Persistent User Preferences**: Remember settings across conversations
- **Context-Aware Responses**: Better personalization
- **Session Management**: Handle complex workflows
- **Task Tracking**: Remember what user has done before

In [1]:
# Import required modules
from typing_extensions import Annotated
from langchain_ollama import ChatOllama
from langchain.agents import create_agent, AgentState
from langgraph.graph.message import add_messages
import tools

## Understanding Default AgentState

First, let's understand what the default AgentState provides:

In [2]:
print("=== Custom State for Memory Management ===")

# Define custom state schema
class CustomAgentState(AgentState):
    """Extended agent state with comprehensive memory management."""
    
    # Required: conversation history (from base AgentState)
    messages: Annotated[list, add_messages]
    
    # Custom: user preferences and session data
    user_preferences: dict = {}
    session_data: dict = {}

print("✓ CustomAgentState defined with custom tracking:")
print("  - messages: Conversation history (required)")
print("  - user_preferences: User settings and preferences")
print("  - session_data: Temporary session information")

=== Custom State for Memory Management ===
✓ CustomAgentState defined with custom tracking:
  - messages: Conversation history (required)
  - user_preferences: User settings and preferences
  - session_data: Temporary session information


In [3]:
print("=== Creating Agent with Custom State ===")

model = ChatOllama(model="qwen3")

# Create agent with custom state schema
agent = create_agent(
    model,
    tools=[tools.save_user_preference, tools.get_user_preference],
    state_schema=CustomAgentState  # This enables custom state tracking
)

print("✓ Agent created with custom state schema")
print("  The agent can now track user preferences and session data")
print("  State persists throughout the conversation")

=== Creating Agent with Custom State ===
✓ Agent created with custom state schema
  The agent can now track user preferences and session data
  State persists throughout the conversation


## Creating Agent with Custom State

Now let's create an agent that uses our custom state schema:

In [4]:
print("=== Testing Custom State Features - Saving Preferences ===")

# Test 1: Save multiple preferences
user_message = """Please save these user preferences for me:
- explanation_preference: technical explanations and detailed examples
- code_style: verbose with comments
- output_format: structured with examples"""

print(f"User input: {user_message}\n")

result = agent.invoke({
    "messages": user_message,
    "session_data": {
        "session_id": "demo123",
        "start_time": "2024-01-01T10:00:00"
    }
})

print(f"Agent response: {result['messages'][-1].content}\n")
print("=" * 60)

=== Testing Custom State Features - Saving Preferences ===
User input: Please save these user preferences for me:
- explanation_preference: technical explanations and detailed examples
- code_style: verbose with comments
- output_format: structured with examples

Agent response: <think>
</think>

All user preferences have been successfully saved:

- **explanation_preference**: Technical explanations and detailed examples
- **code_style**: Verbose with comments
- **output_format**: Structured with examples

Let me know if you need to retrieve or adjust these preferences later!



In [5]:
print("=== Testing Custom State Features - Reading Preferences ===")

# Test 2: Retrieve saved preferences
user_message = "What preferences do I have saved? Please check explanation_preference, code_style, and output_format."

print(f"User input: {user_message}\n")

result = agent.invoke({
    "messages": result["messages"] + [{"role": "user", "content": user_message}],
    "session_data": result.get("session_data", {})
})

print(f"Agent response: {result['messages'][-1].content}\n")
print("=" * 60)
print("\n✓ Preferences are now persisted to file and can be retrieved across sessions")

=== Testing Custom State Features - Reading Preferences ===
User input: What preferences do I have saved? Please check explanation_preference, code_style, and output_format.

Agent response: <think>
Okay, let me process this. The user asked to check their saved preferences: explanation_preference, code_style, and output_format. I called the get_user_preference function for each of these keys. The responses came back with the values they set earlier. Now I need to present this information clearly.

First, I'll list each preference with its corresponding value. Make sure to format it in a way that's easy to read, maybe using bullet points or a structured list. Also, since the user might be verifying their settings, I should confirm that these are the current values. Maybe add a note at the end offering further assistance if needed. Let me double-check that all three preferences are included and correctly displayed. Alright, time to put it all together.
</think>

Here are your currently s